In [4]:
import pandas as pd
import numpy as np

In [5]:
class Config:
    pass
config = pd.read_pickle('config.pkl')
data_path = config.data_path
feature_path = config.feature_path
normalized_path = config.normalized_path

In [6]:
to_stack_path = '../../kaggleData/JD_logging/to_stack/'
config.to_stack_feature_dict={}

In [7]:
trade_tt = pd.read_csv(data_path+'trade_tt_with_recent_3_log_id.csv')
trade_tt.columns

Index(['rowkey', 'time', 'id', 'from', 'month', 'day', 'weekday', 'hour',
       'day_cycle', 'weekday_cycle', 'hour_cycle', 'recent_login_id_0',
       'recent_login_id_1', 'recent_login_id_2'],
      dtype='object')

In [8]:
config.test_7_index = list(trade_tt[trade_tt['month']==7].index)
pd.to_pickle(config,'config.pkl')

In [9]:
config.test_start_stacking = len(config.train_2_6_index)
pd.to_pickle(config,'config.pkl')

In [10]:
feature_sequence = []
for feature in config.numerical_features:
    feature_sequence.append(pd.read_pickle(normalized_path+feature+'.pkl').reshape(-1,1))
numerical_mat = np.hstack(feature_sequence)[config.train_2_6_index+config.test_7_index]


trade_numerical_features = pd.DataFrame(numerical_mat,columns = config.numerical_features)

### 3sigma method

In [11]:
_3sigma_features = []
for feature in config.numerical_features:
    mean = trade_numerical_features[feature].mean()
    std = trade_numerical_features[feature].std()
    upper = mean+3*std
    lower = mean-3*std
    trade_numerical_features[feature+'_3sigma'] = trade_numerical_features[feature].apply(lambda x : 1 if x> upper else (-1 if x<lower else 0))
    _3sigma_features.append(feature+'_3sigma')

In [12]:
config.to_stack_feature_dict['3sigma_detect'] = _3sigma_features

In [13]:
for feature in _3sigma_features:
    pd.to_pickle(trade_numerical_features[feature],to_stack_path+feature+'.pkl')

### Tukey method

In [14]:
tukey_features = []
for feature in config.numerical_features:
    up_quan = np.percentile(trade_numerical_features[feature],0.75)
    low_quan = np.percentile(trade_numerical_features[feature],0.25)
    iqr = up_quan - low_quan
    upper = up_quan + 1.5*iqr
    lower = low_quan - 1.5*iqr
    trade_numerical_features[feature+'_tukey'] = trade_numerical_features[feature].apply(lambda x : 1 if x> upper else (-1 if x<lower else 0))
    tukey_features.append(feature+'_tukey')

In [15]:
config.to_stack_feature_dict['tukey_detect']  = tukey_features

In [41]:
for feature in tukey_features:
    pd.to_pickle(trade_numerical_features[feature],to_stack_path+feature+'.pkl')

### features for unsupervised outlier detectors

In [19]:
feature_sequence = []
for feature in config.normalized_features:
    feature_sequence.append(pd.read_pickle(normalized_path+feature+'.pkl').reshape(-1,1))
normalized_mat = np.hstack(feature_sequence)[config.train_2_6_index+config.test_7_index]

trade_normalized_features = pd.DataFrame(normalized_mat,columns = config.normalized_features)

### isolation forest

In [44]:
from sklearn.ensemble import IsolationForest

In [45]:
if_detector = IsolationForest(n_estimators = 200, contamination = 0.05, max_features = 0.75,n_jobs = 7,random_state =552)
if_detector.fit(normalized_mat)
trade_normalized_features['if_outlier_detect'] = if_detector.predict(normalized_mat)

### Local Outlier Factor(LOF)

In [46]:
from sklearn.neighbors import LocalOutlierFactor 

In [47]:
lof_detector = LocalOutlierFactor(contamination = 0.05,n_jobs = 7)
trade_normalized_features['lof_outlier_detect'] = lof_detector.fit_predict(normalized_mat)

### one-class svm

In [20]:
from sklearn.svm import OneClassSVM

In [ ]:
one_way_svm_detector= OneClassSVM(random_state =552,verbose=True)
one_way_svm_detector.fit(normalized_mat)
trade_normalized_features['svm_1_detect'] = one_way_svm_detector.predict(normalized_mat,random_state=552)

[LibSVM]

### save the unsupervised outlier detector results

In [16]:
#config.to_stack_feature_dict['unsupervised_detect']  = ['if_outlier_detect','lof_outlier_detect','svm_1_detect']
#for feature in config.to_stack_feature_dict['unsupervised_detect']:
#    pd.to_pickle(trade_normalized_features[feature],to_stack_path+feature+'.pkl')

NameError: name 'trade_normalized_features' is not defined

In [17]:
config.to_stack_feature_dict['unsupervised_detect']  = ['if_outlier_detect','lof_outlier_detect']
pd.to_pickle(config,'config.pkl')